In [1]:
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
import re

In [2]:
RANDOM_SEED = 42

# Setup

In [3]:
pass

# Data

In [4]:
df_train = pd.read_csv('BMW_train_ANSI.csv', encoding = 'ANSI', sep = '|')
df_test = pd.read_csv('test.csv')

#  удаляем записи, где не указана цена
df_train.dropna(subset = ['price'], inplace = True)
# удаляем столбцы, которые не несут информации для модели
df_train.drop(['id', 'URL'], axis = 1, inplace = True)
df_test.drop(['id'], axis = 1, inplace = True)

print(df_train.shape, df_test.shape)

(21598, 23) (3837, 22)


In [5]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3837 entries, 0 to 3836
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              3837 non-null   object 
 1   brand                 3837 non-null   object 
 2   color                 3837 non-null   object 
 3   fuelType              3837 non-null   object 
 4   modelDate             3837 non-null   float64
 5   name                  3837 non-null   object 
 6   numberOfDoors         3837 non-null   float64
 7   productionDate        3837 non-null   float64
 8   vehicleConfiguration  3837 non-null   object 
 9   vehicleTransmission   3837 non-null   object 
 10  engineDisplacement    3837 non-null   object 
 11  enginePower           3837 non-null   object 
 12  description           3837 non-null   object 
 13  mileage               3837 non-null   float64
 14  Комплектация          3837 non-null   object 
 15  Привод               

In [51]:
def preproc_data(df_input):
    df_output = df_input.copy()

# Переведем сведения о мощности и объеме двигателя в числовые данные
# по результатам версии 2, это не сильно помогает: на тесте MAPE уменьшился с 10.78 до 10.75
   
    df_output['enginePower'] = df_input['enginePower'].apply(lambda x: int(x[:-4]))
    df_output['engineDisplacement'] = df_input['engineDisplacement'].apply(lambda x: 0 if x == 'undefined LTR' else 10 * float(x[:-4]))
    
    df_output.drop(['Таможня', 'Состояние'], axis=1, inplace=True)
    
    df_output['Комплектация'] = df_input['Комплектация'].apply(len_configuration_parsing)
    
    return df_output

def len_configuration_parsing(txt):
    configuration = []
    pattern = r'\"values\":\[(.+?)\]}'
    for txt_elem in re.findall(pattern, txt):
        pattern_txt_elem = r'\"(.+?)\"'
        for config_item in re.findall(pattern_txt_elem, txt_elem):
            configuration.append(config_item)
    return len(configuration)

In [52]:
train = preproc_data(df_train)
test = preproc_data(df_test)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21598 entries, 0 to 21765
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              21598 non-null  object 
 1   brand                 21598 non-null  object 
 2   color                 21598 non-null  object 
 3   fuelType              21598 non-null  object 
 4   modelDate             21598 non-null  float64
 5   name                  21595 non-null  object 
 6   numberOfDoors         21598 non-null  float64
 7   productionDate        21598 non-null  float64
 8   vehicleConfiguration  21598 non-null  object 
 9   vehicleTransmission   21598 non-null  object 
 10  engineDisplacement    21598 non-null  float64
 11  enginePower           21598 non-null  int64  
 12  description           21595 non-null  object 
 13  mileage               21595 non-null  float64
 14  Комплектация          21598 non-null  int64  
 15  Привод             

In [53]:
train['Комплектация'].iloc[32]

18